## Data Preparation

In [1]:
# import pyhton libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# load dataset
df = pd.read_csv('TravelInsurancePrediction.csv')
df.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,0,31,Government Sector,Yes,400000,6,1,No,No,0
1,1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0


In [3]:
# check size of dataframe
df.shape

(1987, 10)

In [4]:
# delete the Unnamed column 
del df['Unnamed: 0']

In [5]:
df.head(3)

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1


In [6]:
# statistical summary
df.describe()

,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1987.000000,1.987000e+03,1987.000000,1987.000000,1987.000000
mean,29.650226,9.327630e+05,4.752894,0.277806,0.357323
std,2.913308,3.768557e+05,1.609650,0.448030,0.479332
min,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [7]:
# checking that there no null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Age                  1987 non-null   int64 
 1   Employment Type      1987 non-null   object
 2   GraduateOrNot        1987 non-null   object
 3   AnnualIncome         1987 non-null   int64 
 4   FamilyMembers        1987 non-null   int64 
 5   ChronicDiseases      1987 non-null   int64 
 6   FrequentFlyer        1987 non-null   object
 7   EverTravelledAbroad  1987 non-null   object
 8   TravelInsurance      1987 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 139.8+ KB


In [8]:
# change fields name to small letters and replace white spaces with underscore
df.columns = df.columns.str.lower().str.replace(' ','_')

# select all the categorical variables and make a list
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

# loop through the list change the text to small letters and replace white spaces with underscore
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

## Exploratory Data Analysis

In [13]:
# number ofcustomers - 0 for does who didn't buy the travel insurance package while 1 for those who did
df.travelinsurance.value_counts()

0    1277
1     710
Name: travelinsurance, dtype: int64

In [14]:
# defining the average for the travel insurance
avg = df.travelinsurance.mean()
round(avg, 3)

0.357

In [15]:
# split the train dataset into categorical variable
categorical = list(df.dtypes[df.dtypes == 'object'].index)

# split the train dataset into numerical variable
numerical = ['age', 'annualincome', 'familymembers', 'chronicdiseases']

## Feature Importance

In [16]:
# import IPython display library
from IPython.display import display

In [17]:
# loof through the entire categorical dataset to determine the churn and risk rate
for col in categorical:
    df_group = df.groupby(by=col).travelinsurance.agg(['mean'])
    df_group['diff'] = df_group['mean'] - avg
    df_group['risk'] = (df_group['mean'] / avg) - 1
    display(df_group)

,mean,diff,risk
employment_type,,,
government_sector,0.245614,-0.111709,-0.312627
private_sector/self_employed,0.402258,0.044936,0.125757


,mean,diff,risk
graduateornot,,,
no,0.335593,-0.021729,-0.060812
yes,0.361111,0.003789,0.010603


,mean,diff,risk
frequentflyer,,,
no,0.300000,-0.057323,-0.160423
yes,0.573141,0.215819,0.603989


,mean,diff,risk
evertravelledabroad,,,
no,0.256378,-0.100944,-0.282502
yes,0.784211,0.426888,1.194685


## Feature Importance: Mutual Information

In [18]:
# import mutual info score library
from sklearn.metrics import mutual_info_score

In [19]:
# define function for categorical variables to determine feature importance
def calculate_mi(series):
    return mutual_info_score(series, df.travelinsurance)

df_mi = df[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending = False).to_frame(name = 'MI')

display(df_mi.head())

,MI
evertravelledabroad,0.091751
frequentflyer,0.025983
employment_type,0.011351
graduateornot,0.000181


## Feature Importance: Correlation

In [20]:
# determine the correlation between numerical values and travelinsurance
df[numerical].corrwith(df.travelinsurance).to_frame('correlation')

,correlation
age,0.061060
annualincome,0.396763
familymembers,0.079909
chronicdiseases,0.018190


In [21]:
# group travelinsurance by the numerical values
df.groupby(by = 'travelinsurance')[numerical].mean()

,age,annualincome,familymembers,chronicdiseases
travelinsurance,,,,
0,29.517619,8.212999e+05,4.657009,0.271731
1,29.888732,1.133239e+06,4.925352,0.288732
